In [1]:
import csv
import json
import re
import collections
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pprint
pp=pprint.PrettyPrinter(indent=4)

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/My Drive/추천

In [ ]:
#ls

In [4]:
csvfile = open('query_results.txt', 'r') # 2랑 차이는 뭘까 ? 2는 주피터에서 실행이 안된다!
fieldnames = ("product_NAME","tag_NAME")
reader = csv.DictReader(csvfile, fieldnames)
print(reader)

읽은 파일을 저장하기

In [5]:
product_tag_list = collections.defaultdict(list)

for row in reader:
#     print(row['product_NAME'])
    #print(row['product_NAME'], '\n', row['tag_NAME'])
#     col_name=['name','keywords']

    product_tag_list[row['product_NAME']].append(row['tag_NAME'])

In [7]:
print(len(product_tag_list))
type(product_tag_list)

2565


collections.defaultdict

In [ ]:
type(product_tag_list)

In [ ]:
product_tag_list

In [8]:
dic = dict(product_tag_list)

In [ ]:
for d in dic:
  print(d) # dict 의 key 가 출력되겠지 ?

Data Cleaning: 태그, 상품 설명 정제하기

In [9]:
text_to_remove=['/','.','&','!','(',')','g','kg','~','%','1','2','의','가','안','으로','습니다','는','를','ㅁ','이','없이','어'] #제거할 태그(특수문자, 단위, 조사 등)

dic2={}

for key,val in dic.items():
    
  for k in key.split(): # key 공백으로 분리해서 키워드에 추가했다 ?
    val.append(k)
    
  val=list(set(val)) #중복 제거

  for i in range(len(text_to_remove)): # 제거할 키워드 루프 돌면서
    tmp=text_to_remove[i]
    if tmp in val: # 제거할 키워드라면 지운다
      val.remove(tmp)
    
  for n in val: # 키워드 루프돌면서
    s=re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', n) # re.sub : 패턴에 일치하는 문자열 다른 문자로 바꾸기 : 키워드에 '' 로 변경
    t=re.sub('\d+g','',s) # 한번 더 수행
    #print(n,t)
    
    val.remove(n) # 오리지널 지우고
    val.append(t) # 변환한 문자 더하기
    
  for n in val: # 다시 키워드 루프돌면서
    if n.isdigit(): # 숫자라면 지운다
      val.remove(n)
      #print(n)
      continue
      #print(n)
    if 'g' in n: # 그램 단위도 지운다
      val.remove(n)
      continue
        
  if '' in val: # '' 키워드 지운다
    val.remove('')
    
  val.sort() # 키워드 리스트 오름차 정렬한다
  #print(key,val)
    
  dic2[key]=val # 다른 dict 변수에 새로 저장한다
  #print(val)

In [10]:
 doc_list=[]
 for key, val in dic2.items(): # 아이템:키워드 루프돈다 - list 에 추가
    doc_list_row = []
    col_name = ['name','keywords']
    
    doc_list_row.append(key)
    doc_list_row.append(val)
    
    doc_list.append(doc_list_row)
#     print("key = {key}, value={value}".format(key=key,value=val))

In [11]:
doc_list

[['1등급 구운란 10구',
  ['10구',
   '1등급',
   '가마',
   '계란',
   '구',
   '구운',
   '구운란',
   '굽',
   '등급',
   '란',
   '만',
   '선별',
   '에',
   '을',
   '입',
   '팩',
   '하']],
 ['6년근 홍삼이 들어간 삼계탕 재료 140g',
  ['6년근',
   '건강',
   '근',
   '년',
   '더욱',
   '들어가',
   '들어간',
   '봉',
   '삼계',
   '삼계탕',
   '재료',
   '지',
   '탕',
   '하',
   '홍삼',
   '홍삼이']],
 ['GAP 가지 2입', ['2입', 'GAP', '가지', '고', '낮', '봉', '수분', '입', '칼로리', '풍부하']],
 ['GAP 네불라 토마토 500g', ['GAP', '고', '네불라', '단단하', '새콤달콤하', '토마토']],
 ['GAP 다다기 오이 7입',
  ['7입', 'GAP', '과', '기', '다다기', '빛', '생', '식감', '오이', '입', '청량', '초록', '하']],
 ['GAP 대봉 반건시 4입',
  ['4입',
   'GAP',
   '감',
   '곶',
   '과',
   '대봉',
   '매력',
   '반건',
   '반건시',
   '번',
   '시',
   '에',
   '을',
   '입',
   '한',
   '홍시']],
 ['GAP 대봉감말랭이 300g',
  ['GAP', '간식', '감말랭이', '고', '달큰하', '대봉', '대봉감말랭이', '빛', '주홍', '쫀득하']],
 ['GAP 대저 짭짤이 토마토',
  ['GAP', '대저', '제철', '지금', '짭잘달콤새콤', '짭짤', '짭짤이', '토마토', '팩']],
 ['GAP 대저 짭짤이 토마토 2.5kg',
  ['GAP',
   '게',
   '넉넉하',
   '달콤',
   '대저',
   '맛',
  

In [ ]:
#df = pd.DataFrame.from_dict(dic2, orient='index')

데이터프레임 형태로 변경

In [12]:
keyword_df = pd.DataFrame(doc_list, columns=col_name)
keyword_df

,name,keywords
0,1등급 구운란 10구,"[10구, 1등급, 가마, 계란, 구, 구운, 구운란, 굽, 등급, 란, 만, 선별..."
1,6년근 홍삼이 들어간 삼계탕 재료 140g,"[6년근, 건강, 근, 년, 더욱, 들어가, 들어간, 봉, 삼계, 삼계탕, 재료, ..."
2,GAP 가지 2입,"[2입, GAP, 가지, 고, 낮, 봉, 수분, 입, 칼로리, 풍부하]"
3,GAP 네불라 토마토 500g,"[GAP, 고, 네불라, 단단하, 새콤달콤하, 토마토]"
4,GAP 다다기 오이 7입,"[7입, GAP, 과, 기, 다다기, 빛, 생, 식감, 오이, 입, 청량, 초록, 하]"
...,...,...
2560,홍고추 80g,"[고추, 궁금하, 맛, 빨갛, 요리, 위하, 홍, 홍고추]"
2561,황태포 5미,"[5미, 과, 만들, 미, 바다, 바람, 볕, 선물, 햇, 황태포]"
2562,황토방 숙성 건조 감말랭이 55g,"[간식, 감말랭이, 건조, 방, 봉, 손이가, 숙성, 영양, 자꾸만, 쪽득쫀득, 황..."
2563,흙생강 100g,"[손질, 신선하, 않, 알싸하, 지, 하, 흙생강]"


keywords 리스트 원소들을 공백으로 이어붙임

In [13]:
keyword_df['keywords'] = keyword_df['keywords'].apply(lambda x: (' ').join(x)) # DF: 공백으로 이어붙임
keyword_df # 왜 2565 뿐일까 ? - 만약 기존 아이템 개수랑 다르면, 주피터에서 etri api 직접 이용하는 방식으로 수정하기

,name,keywords
0,1등급 구운란 10구,10구 1등급 가마 계란 구 구운 구운란 굽 등급 란 만 선별 에 을 입 팩 하
1,6년근 홍삼이 들어간 삼계탕 재료 140g,6년근 건강 근 년 더욱 들어가 들어간 봉 삼계 삼계탕 재료 지 탕 하 홍삼 홍삼이
2,GAP 가지 2입,2입 GAP 가지 고 낮 봉 수분 입 칼로리 풍부하
3,GAP 네불라 토마토 500g,GAP 고 네불라 단단하 새콤달콤하 토마토
4,GAP 다다기 오이 7입,7입 GAP 과 기 다다기 빛 생 식감 오이 입 청량 초록 하
...,...,...
2560,홍고추 80g,고추 궁금하 맛 빨갛 요리 위하 홍 홍고추
2561,황태포 5미,5미 과 만들 미 바다 바람 볕 선물 햇 황태포
2562,황토방 숙성 건조 감말랭이 55g,간식 감말랭이 건조 방 봉 손이가 숙성 영양 자꾸만 쪽득쫀득 황토 황토방
2563,흙생강 100g,손질 신선하 않 알싸하 지 하 흙생강


In [14]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
keyword_mat = count_vect.fit_transform(keyword_df['keywords']) # 피처 벡터화된 행렬 추출

In [ ]:
keyword_mat.shape

피처 벡터화된 행렬에 코사인 유사도를 적용

In [15]:
keyword_sim = cosine_similarity(keyword_mat, keyword_mat)
keyword_sim[0,:] # 첫행 출력해보기

array([1., 0., 0., ..., 0., 0., 0.])

In [ ]:
for i in range(10): # 모든 행 출력하기
  print(keyword_sim[i,:])

In [ ]:
keyword_sim.shape

유사도가 내림차 순으로 정렬된 행렬의 인덱스를 가져오기

In [16]:
keyword_sim_sorted_index = keyword_sim.argsort()[:,::-1]
keyword_sim_sorted_index[:,0] # 모든 행(모든 아이템)에 대해 유사도가 가장 놓은 아이템(첫 열) 인덱스를 추출한다

array([   0,    1,    2, ..., 2562, 2563, 2564], dtype=int64)

유사도가 높은 5순위 상품들의 인덱스

In [17]:
index_list = keyword_sim_sorted_index[:,1].tolist() # 0은 자기자신, 1은 그 다음 유사도 높은 아이템 인덱스 : 5개
index_list2 = keyword_sim_sorted_index[:,2].tolist() 
index_list3 = keyword_sim_sorted_index[:,3].tolist() 
index_list4 = keyword_sim_sorted_index[:,4].tolist() 
index_list5 = keyword_sim_sorted_index[:,5].tolist() 

type(index_list)

list

In [ ]:
#pp.pprint(index_list2)

In [ ]:
len(index_list)


최초 DataFrame 은 keyword_df.

keyword_df.iloc[숫자]

In [18]:
# 유사 아이템 저장할 default dict
sim_item_dict = collections.defaultdict(list)
for i in range(len(index_list)): # 범위 [0,len(index_list))
    # keyword_df.iloc[i]['name'] # 판다스 접근? .? [컬럼명?] : 컬럼명으로 접근 잘된다
    if keyword_df.iloc[i][0]==keyword_df.iloc[index_list[i]][0]:
      continue
    
    
#     print(">>기준 상품:",keyword_df.iloc[i][0])
#     print("키워드:",keyword_df.iloc[i][1]) 
    sim_item_dict[keyword_df.iloc[i][0]].append(keyword_df.iloc[index_list[i]][0])
    sim_item_dict[keyword_df.iloc[i][0]].append(keyword_df.iloc[index_list2[i]][0])
    sim_item_dict[keyword_df.iloc[i][0]].append(keyword_df.iloc[index_list3[i]][0])
    sim_item_dict[keyword_df.iloc[i][0]].append(keyword_df.iloc[index_list4[i]][0])
    sim_item_dict[keyword_df.iloc[i][0]].append(keyword_df.iloc[index_list5[i]][0])
    
#     print(">유사 상품1:", keyword_df.iloc[index_list[i]][0] ) 
#     print("키워드:", keyword_df.iloc[index_list[i]][1]) 

#     print(">유사 상품2:", keyword_df.iloc[index_list2[i]][0] ) 
#     print("키워드:", keyword_df.iloc[index_list2[i]][1]) 

#     print(">유사 상품3:", keyword_df.iloc[index_list3[i]][0] ) 
#     print("키워드:", keyword_df.iloc[index_list3[i]][1]) 

#     print(">유사 상품4:", keyword_df.iloc[index_list4[i]][0] ) 
#     print("키워드:", keyword_df.iloc[index_list4[i]][1]) 

#     print(">유사 상품5:", keyword_df.iloc[index_list5[i]][0] ) 
#     print("키워드:", keyword_df.iloc[index_list5[i]][1]) 
    
#     print("===================================================================")
    #keyword_df.iloc[index_list[i]]['name']

In [ ]:
#### 기존 아이템 가져오기
- 기존 아이템 루프돌면서, 유사아이템 리스트 끝 열에 이어붙이기

In [28]:
df_from_excel = pd.read_excel('./KurlyProduct.xlsx', sheet_name = 'KurlyProduct')

df_from_excel

,index,category,no,name,price,desc,img
0,0,907001,51700,무농약 저민 마늘 50g,1980,편썰기로 요리에 간편하게 이용하는 저민마늘,https://img-cf.kurly.com/shop/data/goods/15892...
1,1,907001,52650,국산 햇 완두콩 500g,5400,알알이 영근 풋풋함,https://img-cf.kurly.com/shop/data/goods/15888...
2,2,907001,51852,[네떼] 유러피안 루꼴라 믹스 200g,4800,색다르게 즐기는 잎채소 믹스(200g/1봉),https://img-cf.kurly.com/shop/data/goods/15877...
3,3,907001,37314,[네떼]베이비 당근 250g,3600,아담한 사이즈에 하나씩 쓰기 좋은 앙증맞은 당근(1팩/250g),https://img-cf.kurly.com/shop/data/goods/15610...
4,4,907001,51487,고춧잎 150g,2200,여리고 고소한 맛,https://img-cf.kurly.com/shop/data/goods/15869...
...,...,...,...,...,...,...,...
4717,4717,918011,11029,[이토바인더리] 드로잉패드 5종,16000,영감을 주는 노트,https://img-cf.kurly.com/shop/data/goods/15178...
4718,4718,918011,33097,[이리자 한복] 날개 보자기 4종,49000,곱게 묶어 전하는 정성,https://img-cf.kurly.com/shop/data/goods/15462...
4719,4719,918011,33100,[이리자 한복] 사면 장식 모시 보자기,57000,한 겹 모시로 감싼 귀한 마음,https://img-cf.kurly.com/shop/data/goods/15462...
4720,4720,918011,33111,[선물세트] 이리자 한복 잣 물림 보자기 2종,55000,잣씨 장식으로 살린 전통의 아름다움,https://img-cf.kurly.com/shop/data/goods/15462...


In [32]:
df_from_excel['sim_items'] = ['' for x in range(len(df_from_excel))] # 유사 아이템 저장할 컬럼 확보하기
# len(df_from_excel) 
df_from_excel

,index,category,no,name,price,desc,img,sim_items
0,0,907001,51700,무농약 저민 마늘 50g,1980,편썰기로 요리에 간편하게 이용하는 저민마늘,https://img-cf.kurly.com/shop/data/goods/15892...,
1,1,907001,52650,국산 햇 완두콩 500g,5400,알알이 영근 풋풋함,https://img-cf.kurly.com/shop/data/goods/15888...,
2,2,907001,51852,[네떼] 유러피안 루꼴라 믹스 200g,4800,색다르게 즐기는 잎채소 믹스(200g/1봉),https://img-cf.kurly.com/shop/data/goods/15877...,
3,3,907001,37314,[네떼]베이비 당근 250g,3600,아담한 사이즈에 하나씩 쓰기 좋은 앙증맞은 당근(1팩/250g),https://img-cf.kurly.com/shop/data/goods/15610...,
4,4,907001,51487,고춧잎 150g,2200,여리고 고소한 맛,https://img-cf.kurly.com/shop/data/goods/15869...,
...,...,...,...,...,...,...,...,...
4717,4717,918011,11029,[이토바인더리] 드로잉패드 5종,16000,영감을 주는 노트,https://img-cf.kurly.com/shop/data/goods/15178...,
4718,4718,918011,33097,[이리자 한복] 날개 보자기 4종,49000,곱게 묶어 전하는 정성,https://img-cf.kurly.com/shop/data/goods/15462...,
4719,4719,918011,33100,[이리자 한복] 사면 장식 모시 보자기,57000,한 겹 모시로 감싼 귀한 마음,https://img-cf.kurly.com/shop/data/goods/15462...,
4720,4720,918011,33111,[선물세트] 이리자 한복 잣 물림 보자기 2종,55000,잣씨 장식으로 살린 전통의 아름다움,https://img-cf.kurly.com/shop/data/goods/15462...,


Padnas 사용해볼만한 문법
- titanic_df[ titanic_df['Pclass']==3 ]
- titanic_df['Age_Cat'].apply(lambda x: get_category(x), inplace=True)

In [48]:
def get_sim_items(name): #lambda에 이용할 함수 선언 , sim_item_dict

    return ','.join(sim_item_dict[name])


In [50]:

# for row in df_from_excel: # 열을 루프하는데 ?
#     print(row)
# for row in sim_item_dict:
#     print(row)
#     print(df_from_excel[ df_from_excel['name']==row ].values) # 행을 얻어서 뭐할거야 ?
#     print(type(df_from_excel[ df_from_excel['name']==row ])) # DataFrame
#     print( df_from_excel[ df_from_excel['name']==row ]['name'] ) # ok : 컬럼 위치 확보
    
df_from_excel['sim_items'] = df_from_excel['name'].apply(lambda x: get_sim_items(x))

#     df_from_excel[ df_from_excel['name']==row ]['sim_items'] = ','.join(sim_item_dict[row]) # 잠시만 원하는 결과 맞는지 확인좀
#     print(sim_item_dict[row])

In [51]:
df_from_excel

,index,category,no,name,price,desc,img,sim_items
0,0,907001,51700,무농약 저민 마늘 50g,1980,편썰기로 요리에 간편하게 이용하는 저민마늘,https://img-cf.kurly.com/shop/data/goods/15892...,"무농약 꼭지제거한 마늘 200g,무농약 간편 단호박 2종,무농약 마늘종 400g,무..."
1,1,907001,52650,국산 햇 완두콩 500g,5400,알알이 영근 풋풋함,https://img-cf.kurly.com/shop/data/goods/15888...,"유기농 완두콩 600g (냉동),돌나물 150g,[김구원선생] 전통 두부 300g,..."
2,2,907001,51852,[네떼] 유러피안 루꼴라 믹스 200g,4800,색다르게 즐기는 잎채소 믹스(200g/1봉),https://img-cf.kurly.com/shop/data/goods/15877...,"[네떼] 프릴라이즈 180g,[네떼] 셀러리 스틱 170g,[네떼] 스틱채소 플래터..."
3,3,907001,37314,[네떼]베이비 당근 250g,3600,아담한 사이즈에 하나씩 쓰기 좋은 앙증맞은 당근(1팩/250g),https://img-cf.kurly.com/shop/data/goods/15610...,"[네떼] 1900 하루샐러드 비트&당근 105g,[네떼] 베이비믹스 100g,[네떼..."
4,4,907001,51487,고춧잎 150g,2200,여리고 고소한 맛,https://img-cf.kurly.com/shop/data/goods/15869...,"볶음 피칸 200g,밤고구마 800g/봉,검정강낭콩 500g,오트밀 250g,귀리쌀..."
...,...,...,...,...,...,...,...,...
4717,4717,918011,11029,[이토바인더리] 드로잉패드 5종,16000,영감을 주는 노트,https://img-cf.kurly.com/shop/data/goods/15178...,
4718,4718,918011,33097,[이리자 한복] 날개 보자기 4종,49000,곱게 묶어 전하는 정성,https://img-cf.kurly.com/shop/data/goods/15462...,
4719,4719,918011,33100,[이리자 한복] 사면 장식 모시 보자기,57000,한 겹 모시로 감싼 귀한 마음,https://img-cf.kurly.com/shop/data/goods/15462...,
4720,4720,918011,33111,[선물세트] 이리자 한복 잣 물림 보자기 2종,55000,잣씨 장식으로 살린 전통의 아름다움,https://img-cf.kurly.com/shop/data/goods/15462...,


In [53]:
df_from_excel.to_excel('result.xlsx')

In [54]:
len(df_from_excel)

4722